In [1]:
import numpy as np
import pandas as pd
import random
import sklearn

In [2]:
import sys
sys.path.append('..')


In [3]:
from feature_extract import *
from utils import load_data
from models.lgb_model import lgb_model_for_upload,lgb_model_for_offline_test

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\18707\AppData\Local\Temp\jieba.cache
Loading model cost 0.746 seconds.
Prefix dict has been built successfully.


In [4]:
import os
path0=os.path.abspath(os.path.dirname(os.getcwd()))
path = path0 +'/data/info_dataset/'
path1 = path0+'/data/'

In [5]:
# get feature

In [6]:
person =person_feature(path)
intent_table = person_intent(path)
work_history, work_history_table = work_history(path)
cert_table = certificate(path)
project_history, project_history_table = project_history(path)
job_table = job_table(path)

In [53]:
# load data
train_data, valid_data, test_data =load_data(path1)
del train_data['Unnamed: 0']
del valid_data['Unnamed: 0']
del test_data['Unnamed: 0']

In [8]:
data_all = pd.concat([train_data,valid_data,test_data], ignore_index=True)
candidate_data, job_data = diliver_current(data_all)

## offline

In [9]:
train=get_k_fold_data(train_data,person,intent_table,job_table,project_history_table,work_history_table,candidate_data,job_data)
valid_data =data_merge(valid_data,person,intent_table,job_table,project_history_table,work_history_table,candidate_data,job_data)
valid = get_data(valid_data, train)


In [10]:
train_data.groupby("岗位编号").aggregate({"标签": "mean"}).reset_index()

,岗位编号,标签
0,135144,0.6
1,137045,0.0
2,146798,0.0
3,436321,0.0
4,462715,0.5
...,...,...
1838,44274688,1.0
1839,44285651,1.0
1840,44298679,1.0
1841,44310665,1.0


In [11]:
temp_data= pd.concat([train,valid, test], ignore_index=True)

In [12]:
temp_data=deliver_record_feature(temp_data)

In [46]:
temp_data[:1].values

array([[775040, 5969825, 0, 1, 12, 3.0, 0, 34, 50.0, 7.0, nan, 4.0,
        161.25, 645.0, 2.0, 4, 2.0, 0, 46.0, 1.0, 8, 88, 0.0, 0.0, 1.0,
        0, 281, 0, 43, 0, 0, 0, 0, 10, 24, '品质质检', 35, 43, 6,
        32.17045454545455, 73, 26, 5.542358498152937, 31.0, 30,
        0.17045454545454544, 8.670454545454545, 20, 0, 5.404811800202786,
        10.0, 0, 0.22727272727272727, 2.261904761904762, 5.0, 1.0,
        1.1206608164468053, 2.0, 2.0, 0.34523809523809523, 3.0, 5.0, 1.0,
        2.8284271247461903, 3.0, 1.0, 0.5, 316.5, 988.0, 0.0,
        265.4531701490277, 240.5, 0.0, 0.05952380952380952,
        2.5795454545454546, 4.0, 1.0, 0.9675373515507131, 3.0, 3.0,
        0.5113636363636364, 10.383720930232558, 180.0, 7.0,
        19.05852818504166, 7.0, 7.0, 0.872093023255814, 1,
        0.7386363636363636, 1.0, 0.8863636363636364]], dtype=object)

In [47]:
temp_data.columns

Index(['岗位编号', '求职者编号', '标签', '性别', '工作年限', '最高学历', '应聘者专业', '年龄', '自荐信字数',
       '可到职天数', '项目经验数', '工作经历数', '平均主要业绩字数', '总主要业绩字数', '招聘对象代码', '招聘对象',
       '岗位最低学历', '岗位工作年限', '具体要求字数', '工作地点符合否', '求职者求职数', '岗位求职者数',
       '求职者平均被匹配数', '岗位平均被匹配数', '专业是否符合', '工作年限是否符合', '最近工作岗位', '最近所在行业',
       '当前工作所在地', '汉语语言能力', '英语语言能力', '日语语言能力', '粤语语言能力', '岗位工作地点', '工作地点',
       '岗位类别', '岗位编号_岗位类别_nunique', '岗位编号_应聘者专业_nunique', '岗位编号_工作地点_nunique',
       '岗位编号_年龄_mean', '岗位编号_年龄_max', '岗位编号_年龄_min', '岗位编号_年龄_std',
       '岗位编号_年龄_median', '岗位编号_年龄_mode', '岗位编号_年龄_mode_rate', '岗位编号_工作年限_mean',
       '岗位编号_工作年限_max', '岗位编号_工作年限_min', '岗位编号_工作年限_std', '岗位编号_工作年限_median',
       '岗位编号_工作年限_mode', '岗位编号_工作年限_mode_rate', '岗位编号_工作经历数_mean',
       '岗位编号_工作经历数_max', '岗位编号_工作经历数_min', '岗位编号_工作经历数_std',
       '岗位编号_工作经历数_median', '岗位编号_工作经历数_mode', '岗位编号_工作经历数_mode_rate',
       '岗位编号_项目经验数_mean', '岗位编号_项目经验数_max', '岗位编号_项目经验数_min', '岗位编号_项目经验数_std',
       '岗位编号_项目经验数_median', '岗位编号_项目经验数_mode', 

In [30]:
train.shape,valid.shape,test.shape

((28232, 36), (7059, 36), (70774, 36))

In [13]:
from sklearn.preprocessing import LabelEncoder

for f in temp_data.select_dtypes('object'):
    le = LabelEncoder()
    print(f)
    temp_data[f] = le.fit_transform(temp_data[f].astype('str'))
temp_data["招聘对象"] =le.fit_transform(temp_data["招聘对象"].astype('str'))
temp_data["最近工作岗位"] =le.fit_transform(temp_data["最近工作岗位"].astype('str'))
temp_data["岗位类别"] =le.fit_transform(temp_data["岗位类别"].astype('str'))

应聘者专业
最近工作岗位
最近所在行业
当前工作所在地
语言能力
专业特长
自荐信
工作地点
所在行业
其他说明
招聘职位
对应聘者的专业要求
岗位工作地点
具体要求
汉语语言能力
英语语言能力
粤语语言能力
日语语言能力


In [30]:
# onehot
data_all_feature=feature_process(temp_data)

In [14]:
# none one-hot
data_all_feature=temp_data

In [21]:
data_all_feature.head(1)

,岗位编号,求职者编号,标签,性别,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,最近所在行业,...,岗位编号_可到职天数_max,岗位编号_可到职天数_min,岗位编号_可到职天数_std,岗位编号_可到职天数_median,岗位编号_可到职天数_mode,岗位编号_可到职天数_mode_rate,岗位编号_性别_mode,岗位编号_性别_mode_rate,岗位编号_工作地点符合否_mode,岗位编号_工作地点符合否_mode_rate
0,775040,5969825,0,1,12,3.0,0,34,281,0,...,180.0,7.0,19.058528,7.0,7.0,0.872093,1,0.738636,1.0,0.886364


In [15]:
train0 = data_all_feature[:train.shape[0]]#.标签.value_counts()
valid0 = data_all_feature[train.shape[0]:train.shape[0]+valid.shape[0]]
test0 = data_all_feature[train.shape[0]+valid.shape[0]:]
train_label = train0[["标签"]]
valid_label = valid0[["标签"]]
train=train0[list(train0.columns)[:2]+list(train0.columns)[3:]]
valid=valid0[list(train0.columns)[:2]+list(train0.columns)[3:]]


In [23]:
train0.head()

,岗位编号,求职者编号,标签,性别,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,最近所在行业,...,岗位编号_可到职天数_max,岗位编号_可到职天数_min,岗位编号_可到职天数_std,岗位编号_可到职天数_median,岗位编号_可到职天数_mode,岗位编号_可到职天数_mode_rate,岗位编号_性别_mode,岗位编号_性别_mode_rate,岗位编号_工作地点符合否_mode,岗位编号_工作地点符合否_mode_rate
0,775040,5969825,0,1,12,3.0,0,34,281,0,...,180.0,7.0,19.058528,7.0,7.0,0.872093,1,0.738636,1.0,0.886364
1,799873,6139430,0,1,12,4.0,216,37,86,116,...,180.0,7.0,21.468561,7.0,7.0,0.754717,1,0.957944,1.0,0.957944
2,779698,6083324,0,1,0,1.0,276,31,235,0,...,180.0,7.0,12.886417,7.0,7.0,0.935043,1,0.856661,0.0,0.984823
3,771109,5873085,0,0,6,4.0,373,33,226,0,...,180.0,7.0,11.683337,7.0,7.0,0.892593,0,0.912088,0.0,0.948718
4,823833,6218282,0,0,0,3.0,349,29,12,0,...,180.0,7.0,14.694844,7.0,7.0,0.903654,0,0.891089,0.0,0.980198


In [48]:
import lightgbm
from sklearn.metrics import f1_score
params = {"objective": "binary",
          "learning_rate": 0.05,
          "max_depth": 6,
          "num_leaves": 32,
          "verbose": -1,
          "bagging_fraction": 0.8,
          "feature_fraction": 0.9,
          'subsample':0.85,
          'bagging_freq':1,
          'random_state':2048}
def lgb_model_for_offline_test(train, train_label, valid, valid_label):
    train_set = lightgbm.Dataset(train.values, label=train_label)
    val_set = lightgbm.Dataset(valid.values, label=valid_label)
    
    model = lightgbm.train(train_set=train_set,
                           valid_sets=[val_set],
                           num_boost_round=3000,
                           params=params,
                           feval=lgb_f1_score,
                           early_stopping_rounds=100)
    return model
def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat)  # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat), True

In [49]:
model=lgb_model_for_offline_test(train,train_label,valid,valid_label)

[1]	valid_0's binary_logloss: 0.410349	valid_0's f1: 0
Training until validation scores don't improve for 100 rounds
[2]	valid_0's binary_logloss: 0.381683	valid_0's f1: 0
[3]	valid_0's binary_logloss: 0.358181	valid_0's f1: 0
[4]	valid_0's binary_logloss: 0.338497	valid_0's f1: 0
[5]	valid_0's binary_logloss: 0.32114	valid_0's f1: 0
[6]	valid_0's binary_logloss: 0.30936	valid_0's f1: 0
[7]	valid_0's binary_logloss: 0.295462	valid_0's f1: 0
[8]	valid_0's binary_logloss: 0.283022	valid_0's f1: 0
[9]	valid_0's binary_logloss: 0.274476	valid_0's f1: 0
[10]	valid_0's binary_logloss: 0.266104	valid_0's f1: 0.674118
[11]	valid_0's binary_logloss: 0.256468	valid_0's f1: 0.698614
[12]	valid_0's binary_logloss: 0.24737	valid_0's f1: 0.722537
[13]	valid_0's binary_logloss: 0.239039	valid_0's f1: 0.733595
[14]	valid_0's binary_logloss: 0.23319	valid_0's f1: 0.738135
[15]	valid_0's binary_logloss: 0.22604	valid_0's f1: 0.756727
[16]	valid_0's binary_logloss: 0.2194	valid_0's f1: 0.76605
[17]	valid

[133]	valid_0's binary_logloss: 0.11538	valid_0's f1: 0.864128
[134]	valid_0's binary_logloss: 0.115383	valid_0's f1: 0.864128
[135]	valid_0's binary_logloss: 0.115407	valid_0's f1: 0.864128
[136]	valid_0's binary_logloss: 0.11535	valid_0's f1: 0.863594
[137]	valid_0's binary_logloss: 0.115308	valid_0's f1: 0.863594
[138]	valid_0's binary_logloss: 0.115313	valid_0's f1: 0.863594
[139]	valid_0's binary_logloss: 0.115236	valid_0's f1: 0.863594
[140]	valid_0's binary_logloss: 0.115266	valid_0's f1: 0.863594
[141]	valid_0's binary_logloss: 0.115206	valid_0's f1: 0.863594
[142]	valid_0's binary_logloss: 0.115145	valid_0's f1: 0.864128
[143]	valid_0's binary_logloss: 0.115101	valid_0's f1: 0.863722
[144]	valid_0's binary_logloss: 0.115111	valid_0's f1: 0.864128
[145]	valid_0's binary_logloss: 0.115138	valid_0's f1: 0.863722
[146]	valid_0's binary_logloss: 0.115143	valid_0's f1: 0.863316
[147]	valid_0's binary_logloss: 0.115081	valid_0's f1: 0.862247
[148]	valid_0's binary_logloss: 0.115048	v

[266]	valid_0's binary_logloss: 0.112943	valid_0's f1: 0.868556
[267]	valid_0's binary_logloss: 0.112957	valid_0's f1: 0.868152
[268]	valid_0's binary_logloss: 0.112874	valid_0's f1: 0.869081
[269]	valid_0's binary_logloss: 0.112835	valid_0's f1: 0.869888
[270]	valid_0's binary_logloss: 0.11282	valid_0's f1: 0.868959
[271]	valid_0's binary_logloss: 0.112816	valid_0's f1: 0.869363
[272]	valid_0's binary_logloss: 0.112824	valid_0's f1: 0.869363
[273]	valid_0's binary_logloss: 0.112817	valid_0's f1: 0.870009
[274]	valid_0's binary_logloss: 0.112869	valid_0's f1: 0.871342
[275]	valid_0's binary_logloss: 0.112852	valid_0's f1: 0.870818
[276]	valid_0's binary_logloss: 0.112856	valid_0's f1: 0.869767
[277]	valid_0's binary_logloss: 0.112834	valid_0's f1: 0.868837
[278]	valid_0's binary_logloss: 0.112805	valid_0's f1: 0.868837
[279]	valid_0's binary_logloss: 0.112776	valid_0's f1: 0.868837
[280]	valid_0's binary_logloss: 0.112698	valid_0's f1: 0.868433
[281]	valid_0's binary_logloss: 0.112628	

[396]	valid_0's binary_logloss: 0.112265	valid_0's f1: 0.877095
[397]	valid_0's binary_logloss: 0.112245	valid_0's f1: 0.876572
[398]	valid_0's binary_logloss: 0.112275	valid_0's f1: 0.877095
[399]	valid_0's binary_logloss: 0.11217	valid_0's f1: 0.876572
[400]	valid_0's binary_logloss: 0.112237	valid_0's f1: 0.876164
[401]	valid_0's binary_logloss: 0.112268	valid_0's f1: 0.875756
[402]	valid_0's binary_logloss: 0.112294	valid_0's f1: 0.875872
[403]	valid_0's binary_logloss: 0.112275	valid_0's f1: 0.875872
[404]	valid_0's binary_logloss: 0.112296	valid_0's f1: 0.876394
[405]	valid_0's binary_logloss: 0.112267	valid_0's f1: 0.876279
[406]	valid_0's binary_logloss: 0.112188	valid_0's f1: 0.875872
[407]	valid_0's binary_logloss: 0.112236	valid_0's f1: 0.876279
[408]	valid_0's binary_logloss: 0.112217	valid_0's f1: 0.876801
[409]	valid_0's binary_logloss: 0.11223	valid_0's f1: 0.876279
[410]	valid_0's binary_logloss: 0.112218	valid_0's f1: 0.875756
[411]	valid_0's binary_logloss: 0.112223	v

In [50]:
pd.DataFrame(zip(list(train.columns),model.feature_importance())).sort_values(1,ascending=False)#.to_csv(r'feature_iportance.csv')

,0,1
41,岗位编号平均被匹配数,656
1,求职者编号,615
56,岗位类别平均被匹配数,408
46,应聘者专业平均被匹配数,404
18,自荐信字数,388
...,...,...
111,岗位编号_工作地点符合否_mode,2
69,岗位编号_工作年限_min,1
36,工作年限是否符合,1
106,岗位编号_可到职天数_median,0


In [61]:
ycol = "标签"
feature_names = list(filter(lambda x: x not in [ycol], train.columns))
df_importance = pd.DataFrame({
        'column': feature_names,
        'importance': model.feature_importance(),
    })

In [64]:
df_importance.sort_values("importance").to_csv(r'feature_iportance.csv')

In [51]:
from utils import submission

In [17]:
path2=path0+'/result/'
name ='baseline_offline_2'
submission(path2,model,valid0,name)

..\utils.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["预测打分"] = model.predict(data[data.columns[:-3]].values)


## online

In [52]:
train_data.head()

,Unnamed: 0,岗位编号,求职者编号,标签
0,0,775040,5969825,0
1,1,815337,6212884,0
2,2,770107,5451198,0
3,3,43795661,320873919,1
4,4,798922,5650936,0


In [54]:

# online
data=pd.concat([train_data,valid_data],ignore_index=True)
train=get_k_fold_data(data,person,intent_table,job_table,project_history_table,work_history_table,candidate_data,job_data)
test_data =data_merge(test_data,person,intent_table,job_table,project_history_table,work_history_table,candidate_data,job_data)
test = get_data(test_data, train)
temp_data= pd.concat([train, test], ignore_index=True)
# data_all_feature=feature_process(temp_data)
temp_data=deliver_record_feature(temp_data)
for f in temp_data.select_dtypes('object'):
    le = LabelEncoder()
    print(f)
    temp_data[f] = le.fit_transform(temp_data[f].astype('str'))
temp_data["招聘对象"] =le.fit_transform(temp_data["招聘对象"].astype('str'))
temp_data["最近工作岗位"] =le.fit_transform(temp_data["最近工作岗位"].astype('str'))
temp_data["岗位类别"] =le.fit_transform(temp_data["岗位类别"].astype('str'))





应聘者专业
最近工作岗位
最近所在行业
当前工作所在地
语言能力
专业特长
自荐信
工作地点
所在行业
其他说明
招聘职位
对应聘者的专业要求
岗位工作地点
具体要求
汉语语言能力
英语语言能力
粤语语言能力
日语语言能力


In [54]:
temp_data.head()

,岗位编号,求职者编号,标签,性别,工作年限,最高学历,应聘者专业,年龄,自荐信字数,可到职天数,...,岗位编号_可到职天数_max,岗位编号_可到职天数_min,岗位编号_可到职天数_std,岗位编号_可到职天数_median,岗位编号_可到职天数_mode,岗位编号_可到职天数_mode_rate,岗位编号_性别_mode,岗位编号_性别_mode_rate,岗位编号_工作地点符合否_mode,岗位编号_工作地点符合否_mode_rate
0,775040,5969825,0,1,12,3.0,0,34,50.0,7.0,...,180.0,7.0,19.058528,7.0,7.0,0.872093,1,0.738636,1.0,0.886364
1,799873,6139430,0,1,12,4.0,216,37,148.0,7.0,...,180.0,7.0,21.468561,7.0,7.0,0.754717,1,0.957944,1.0,0.957944
2,779698,6083324,0,1,0,1.0,276,31,65.0,7.0,...,180.0,7.0,12.886417,7.0,7.0,0.935043,1,0.856661,0.0,0.984823
3,771109,5873085,0,0,6,4.0,373,33,39.0,7.0,...,180.0,7.0,11.683337,7.0,7.0,0.892593,0,0.912088,0.0,0.948718
4,823833,6218282,0,0,0,3.0,349,29,NaN,7.0,...,180.0,7.0,14.694844,7.0,7.0,0.903654,0,0.891089,0.0,0.980198


In [55]:
temp_data.head()

,岗位编号,求职者编号,标签,性别,工作年限,最高学历,应聘者专业,年龄,最近工作岗位,最近所在行业,...,岗位编号_可到职天数_max,岗位编号_可到职天数_min,岗位编号_可到职天数_std,岗位编号_可到职天数_median,岗位编号_可到职天数_mode,岗位编号_可到职天数_mode_rate,岗位编号_性别_mode,岗位编号_性别_mode_rate,岗位编号_工作地点符合否_mode,岗位编号_工作地点符合否_mode_rate
0,775040,5969825,0,1,12,3.0,0,34,281,0,...,180.0,7.0,19.058528,7.0,7.0,0.872093,1,0.738636,1.0,0.886364
1,799873,6139430,0,1,12,4.0,216,37,86,116,...,180.0,7.0,21.468561,7.0,7.0,0.754717,1,0.957944,1.0,0.957944
2,779698,6083324,0,1,0,1.0,276,31,235,0,...,180.0,7.0,12.886417,7.0,7.0,0.935043,1,0.856661,0.0,0.984823
3,771109,5873085,0,0,6,4.0,373,33,226,0,...,180.0,7.0,11.683337,7.0,7.0,0.892593,0,0.912088,0.0,0.948718
4,823833,6218282,0,0,0,3.0,349,29,12,0,...,180.0,7.0,14.694844,7.0,7.0,0.903654,0,0.891089,0.0,0.980198


In [74]:
def lgb_model_for_upload(data, label):
    model = lightgbm.train(train_set=lightgbm.Dataset(data.values, label.values),
                           num_boost_round=800,
                           params=params)

    return model

In [75]:
data_all_feature  =temp_data
#train,train_label,valid,valid_label
train0 = data_all_feature[:train.shape[0]]#.标签.value_counts()
test0 = data_all_feature[train.shape[0]:]
train_label = train0["标签"]
train=train0[list(train0.columns)[:2]+list(train0.columns)[3:]]


In [76]:
model = lgb_model_for_upload(train, train_label)

In [77]:
# from utils import submission
def submission(path, model, test0, name):
    test0["预测打分"] = model.predict(test0[list(test0.columns)[:2]+list(test0.columns)[3:]].values)
    test_data = test0.sort_values("预测打分", ascending=False, ignore_index=True)
    test_data["预测"] = 0
    test_data.loc[:int(0.15 * len(test_data)), ["预测"]] = 1

    submit = test_data[["岗位编号", "求职者编号", "预测"]]
    submit.columns = ["RECRUIT_ID", "PERSON_ID", "LABEL"]
    submit.to_csv(path + name + ".csv", index=False)
path2=path0+'/result/'
name ='online_add_cross_feature_800_v2'
submission(path2,model,test0,name)

d:\software\python\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [153]:
model.predict(test0[list(test0.columns)[:2]+list(test0.columns)[3:]].values)

array([0.705611  , 0.81274684, 0.29015462, ..., 0.99990204, 0.9995563 ,
       0.99997708])

In [149]:
test0[list(test0.columns)[:2]+list(test0.columns)[3:]]

,岗位编号,求职者编号,性別,工作年限,最高学历,应聘者专业,年龄,自荐信字数,可到职天数,项目经验数,...,岗位平均被匹配数,专业是否符合,工作年限是否符合,最近工作岗位,最近所在行业,当前工作所在地,汉语语言能力,英语语言能力,日语语言能力,粤语语言能力
35291,135144,996499,1,15,1.0,80,34,3.0,NaN,NaN,...,0.428571,0.0,0,34,0,12,0,0,0,2
35292,135144,3301030,1,12,2.0,112,37,31.0,NaN,NaN,...,0.428571,0.0,0,9,0,43,8,0,0,2
35293,135144,3404860,1,0,2.0,0,32,NaN,7.0,NaN,...,0.428571,1.0,1,1,60,43,0,0,0,0
35294,135144,4155520,1,11,3.0,0,32,169.0,7.0,NaN,...,0.428571,1.0,0,1,60,43,8,0,0,2
35295,135144,4211720,1,12,1.0,211,30,37.0,7.0,NaN,...,0.428571,0.0,0,34,0,43,8,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106060,44350657,321173912,1,5,4.0,377,26,109.0,7.0,1.0,...,1.000000,0.0,1,277,38,19,0,0,0,0
106061,44350657,321377962,1,0,4.0,292,28,152.0,7.0,NaN,...,1.000000,0.0,0,34,0,43,0,0,0,0
106062,47578782,319583916,0,0,4.0,302,28,141.0,7.0,NaN,...,NaN,0.0,0,34,0,70,0,0,0,0
106063,47649893,6079047,1,7,4.0,0,29,153.0,7.0,NaN,...,1.000000,1.0,1,255,82,43,0,0,0,0


In [21]:
# baseline offline: 0.8521
# online  ：0.8544

In [78]:
import numpy as np
ls=[np.array([1,2,3]),np.array([4,5,6])]

In [83]:
sum(ls)

array([5, 7, 9])

* one-hot +岗位编号	求职者编号 0.8633      ONLINE:0.8653
* no one hot +岗位编号	求职者编号 0.865602      ONLINE:0.8663
* 添加招聘岗位投递记录中的求职意向岗位，求职者专业，（年龄、工作经历数、项目经验数、可到职天数）统计指标、工作地点符合占比特征
* 0.8665 online：0.86748
* 添加k-折交叉的标签特征：offline:0.8772  online: 0.8711